In [1]:
'''These are import statements that are used to make the code work from Python Libraries of code that provide python tools utilized below. 
DO NOT EDIT UNLESS AUTHORIZED.
There are a couple more than necessary, but are included still incase any edits are needed in the future that might need these libraries.'''
import pandas as pd
import numpy as np
import ssl
import urllib.request, urllib.parse, urllib.error
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import requests
from bs4 import BeautifulSoup
import glob
import os.path
import re
from pandas.errors import EmptyDataError
from datetime import date



In [2]:
'''EDIT WITH THE PATH OF THE TRACKING DOCUMENT ON THE COMPUTER WHERE THIS SCRIPT IS RAN.. It is essential that it is formated the same as the 
one Rochelle previously provided with the name of the species to be used as search queries on the Atlas Collection in the first line and 
that this information is present on the sheet titled "Listed".

When editing, insert only the path of the file where this information can be found in between the quotation marks inside the parentheses.
Similarly, if the species name exists on a sheet with a different name than "LISTED" in the first line then please insert that sheet name
also in between the quotation marks after it says sheet_name = '''
sourcesheet = pd.read_excel(r"C:\Users\justi\Downloads\GLANSIS Spp. Tracking.xlsx", sheet_name = "Listed")


In [3]:
'''This segment of code will simply generate a list of names that will be searched. DO NOT EDIT.'''
namelist= []
for name in sourcesheet['Scientific Name']:
    namelist.append(name)

In [4]:
'''This is the key part of the script. This is where the web scraping happens. It should run automatically. DO NOT EDIT.'''

atlas_url = "https://bellatlas.umn.edu/collections/harvestparams.php"
#setting up driver for Selenium
driver = webdriver.Firefox() #???Should I write a script for multiple Browsers?

emptyresults = 0
masterdataframe = pd.DataFrame()
driver.maximize_window()
scraped_dataframe = pd.DataFrame()
scraped_dataframe.reset_index(drop=True)
for speciesname in namelist[:]:
    driver.get(atlas_url) # Goes to atlas
    namebox = driver.find_element(by = By.XPATH, value = '//*[@id="taxa"]')
    searchbutton = driver.find_element(by = By.XPATH, value='/html/body/table/tbody/tr/td/div[4]/form/div[2]/div[1]/button') #selects the advanced search button based off the pulled tag from right clicking the inspected element
    time.sleep(.5)
    namebox.clear()
    namebox.send_keys(speciesname)
    searchbutton.click()

    resultcount = driver.find_element(by=By.XPATH, value='/html/body/table/tbody/tr/td/div[4]/div/div[2]/div/div[4]/div[3]').text
    resultcountnumberlist= list(map(int, list(filter(lambda x: x!= '', re.findall(r'\d*', re.search(r'\d* of \d*', resultcount).group()))))) #first number in list is the total results displayed on the page, the second number is the total results

#if there are not results
    if resultcountnumberlist[1] == 0:
        emptyresults+=1
        continue
#continue to next search
    
#if there are results
    windowbefore = driver.window_handles[0]
    downloadbutton = driver.find_element(by=By.XPATH, value='/html/body/table/tbody/tr/td/div[4]/div/div[2]/div/div[1]/form/button')
    downloadbutton.click()
    windowafter = driver.window_handles[1]
    driver.switch_to.window(windowafter)
    time.sleep(2)
    driver.find_element(by=By.XPATH, value='/html/body/div[1]/div[2]/form/fieldset/table/tbody/tr[4]/td[2]/div/input[2]').click() #utf8 encoding
    driver.find_element(by=By.XPATH, value='/html/body/div[1]/div[2]/form/fieldset/table/tbody/tr[5]/td[2]/div/input').click() #uncompresses download

    downloadbutton=driver.find_element(by=By.NAME, value= "submitaction") #OVERWRITES THE PREVIOUS DOWNLOAD BUTTON
    downloadbutton.click()

    folder_path = r'C:\Users\justi\Downloads'
    file_type = r'\*CSV'
    files = glob.glob(folder_path + file_type)
    max_file = max(files, key=os.path.getctime)

    try:
        downloadeddata = pd.read_csv(max_file, encoding='utf-8')
    except:
        try:
            print('Empty Data Error Occured on file ' + max_file)
            downloadeddatafile = open(max_file, 'r')
            downloadeddata = pd.read_csv(downloadeddatafile, encoding='utf-8')
            downloadeddatafile.close()
            continue
        except:
            print("Skipping to next interation")
            continue
        
    
    if masterdataframe.empty:
        masterdataframe = pd.DataFrame(data=downloadeddata)
    else:
        masterdataframe = masterdataframe.append(pd.DataFrame(data = downloadeddata))
    driver.switch_to.window(windowbefore)
    os.remove(max_file)
print('During the search, there were ' + str(emptyresults)+ ' searches with no results.')
    

During the search, there were 1 searches with no results.


In [5]:
'''This creates a file of the entire scraped results. No information is omitted. EDIT WITH THE PATH WHERE THIS CSV FILE SHOULD BE STORED
ON THE COMPUTER WHERE THIS SCRIPT IS RAN.'''
file= masterdataframe.to_csv(r'C:\Users\justi\Downloads\UMinnAtlasScrapeResults' + str(date.today()) +  '.csv', encoding='utf-8')

In [6]:
'''This extracts information from the entire scraped results and inserts them into the bulk upload template. EDIT WITH THE PATH OF WHERE
THE BULK UPLOAD TEMPLATE IS ON THE COMPUTER WHERE THIS SCRIPT IS RAN.'''
clienttemplate = pd.read_excel(r"C:\Users\justi\Downloads\GLANSISSpecimenBulkUploadTemplateTEST.xlsx").drop('Unnamed: 0', axis=1)
clienttemplate[['Genus*', 'Species*', 'Latitude*', 'Longitude*', 'Year*', 'Month', 'Day', 'Collectors', 'Reference ID*', 'Reference 3', 'Number collected/observed', 'Locality*'  ]] = masterdataframe[['genus', 'specificEpithet', 'decimalLatitude', 'decimalLongitude', 'year', 'month', 'day', 'recordedBy', 'id', 'references', 'individualCount', 'locality']]
clienttemplate['Source*'] = np.where(clienttemplate['Latitude*'].isna()|clienttemplate['Longitude*'].isna(), '', 'reported')
clienttemplate['Accuracy*'] = np.where(clienttemplate['Source*'] == 'reported', 'accurate', '') #
clienttemplate['Reference 2']= atlas_url 
clienttemplate['Record Type*'] = np.where(masterdataframe['collectionCode'].isna(), '', 'specimen')
clienttemplate['Year Accuracy of specimen*'] = masterdataframe['year']


In [7]:
'''This will save the bulk upload template with the scraped data in it to a csv file. UPLOAD WITH 
THE PATH OF WHERE THIS FILE SHOULD BE SAVED'''
finalBulkUploadSheet = clienttemplate.to_csv(r'C:\Users\justi\Downloads\UMinnAtlasBulkUploadSheet' + str(date.today())+ '.csv', encoding='utf-8')